# Ollama ile Gelişmiş YFinance Tool Entegrasyonu

Bu not defterinde Ollama LLM ile Python ortamında yfinance tabanlı bir tool'un profesyonel seviyede entegrasyonu, tool çağrılarının otomatik işlenmesi ve sonuçların görselleştirilmesi gösterilmektedir.

In [1]:
# Gerekli kütüphanelerin yüklenmesi ve içe aktarılması
import sys
import importlib
from importlib import util

def pip_install(package):
    try:
        import subprocess
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
        print(f"✅ {package} başarıyla yüklendi.")
        return True
    except Exception as e:
        print(f"❌ {package} yüklenirken hata oluştu: {e}")
        return False

# Gerekli kütüphanelerin kontrolü ve yüklenmesi
required_packages = ["yfinance", "pandas", "matplotlib", "ollama"]
missing_packages = []

for pkg in required_packages:
    if util.find_spec(pkg) is None:
        print(f"📦 {pkg} yükleniyor...")
        if not pip_install(pkg):
            missing_packages.append(pkg)
    else:
        print(f"✓ {pkg} zaten yüklü.")

if missing_packages:
    print(f"⚠️ Bazı paketler yüklenemedi: {', '.join(missing_packages)}")

# İçe aktarma
try:
    import yfinance as yf
    import pandas as pd
    import matplotlib.pyplot as plt
    import ollama
except ImportError as e:
    print(f"❌ İçe aktarma hatası: {e}")

✓ yfinance zaten yüklü.
✓ pandas zaten yüklü.
✓ matplotlib zaten yüklü.
✓ ollama zaten yüklü.


## Tool Fonksiyonunun Profesyonel Tanımı

- Sembol, periyot ve istenirse kolon seçimi desteklenir.
- Hatalı sembol veya veri yoksa kullanıcıya bilgi verilir.
- Tool parametreleri JSON Schema ile tanımlanır.

In [2]:
def get_stock_data(symbol: str, period: str = "5d", columns: list = None) -> dict:
    """Belirtilen sembol ve periyotta hisse verisi döndürür."""
    try:
        df = yf.download(symbol, period=period)
        if df.empty:
            return {"error": f"Veri bulunamadı: {symbol} ({period})"}
        
        # MultiIndex ise (yf.download bazen böyle döndürür) düzleştir
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
            
        if columns:
            # Kolon isimlerini doğru şekilde eşleştir (büyük/küçük harf duyarlılığını yok say)
            available_cols = {col.lower(): col for col in df.columns}
            requested_cols = []
            missing = []
            
            for col in columns:
                if col.lower() in available_cols:
                    requested_cols.append(available_cols[col.lower()])
                else:
                    missing.append(col)
                    
            if missing:
                return {"error": f"Eksik kolonlar: {missing}. Mevcut kolonlar: {list(df.columns)}"}
            
            df = df[requested_cols]
            
        return {"symbol": symbol, "period": period, "data": df.reset_index().to_dict(orient="records")}
    except Exception as e:
        return {"error": f"Veri çekerken hata oluştu: {e}", "symbol": symbol, "period": period}

In [3]:
# Tool parametre şeması (OpenAI/JSON Schema uyumlu)
yfinance_tool_schema = {
    "name": "yfinance_tool",
    "description": "Belirtilen hisse senedinin geçmiş verilerini getirir.",
    "parameters": {
        "type": "object",
        "properties": {
            "symbol": {"type": "string", "description": "Hisse senedi sembolü (örn: AAPL)"},
            "period": {"type": "string", "description": "Veri periyodu (örn: 5d, 1mo, 1y)", "default": "5d"},
            "columns": {
                "type": "array",
                "items": {"type": "string"},
                "description": "Opsiyonel: Dönmek istenen kolonlar (örn: ['Close', 'Volume'])"
            }
        },
        "required": ["symbol"]
    }
}

In [4]:
def ollama_chat_with_tools(prompt, tools, tool_functions, model="llama3.2:3b"):
    """Ollama ile tool destekli sohbet. Tool çağrısı olursa otomatik çalıştırır."""
    try:
        response = ollama.chat(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            tools=tools
        )
        
        msg = response.get("message", {})
        content = msg.get("content", None) if isinstance(msg, dict) else None
        tool_results = []
        
        # Tool çağrısı varsa otomatik çalıştır
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            for call in msg.tool_calls:
                try:
                    func_name = getattr(call.function, "name", None)
                    args = getattr(call.function, "arguments", {}) or {}
                    
                    # Bilinen tool adlarına eşleştir (Bazen LLM farklı isimler üretebilir)
                    matched_func = None
                    for known_func in tool_functions.keys():
                        if func_name == known_func or func_name.lower() == known_func.lower():
                            matched_func = known_func
                            break
                    
                    # Argümanları düzgün formata dönüştür
                    parsed_args = {}
                    
                    # Model bazen farklı parametre isimleri kullanabilir
                    for key, value in args.items():
                        if key in ["symbol", "s", "ticker", "sembol", "security"]:
                            parsed_args["symbol"] = value
                        elif key in ["period", "p", "time", "n", "days", "süre"]:
                            # Sayısal değer ise gün olarak yorumla
                            if value.isdigit():
                                parsed_args["period"] = f"{value}d"
                            else:
                                parsed_args["period"] = value
                        elif key in ["columns", "fields", "cols", "kolonlar"]:
                            if isinstance(value, list):
                                parsed_args["columns"] = value
                            elif isinstance(value, str) and value.strip():
                                # Virgülle ayrılmış string ise listeye çevir
                                parsed_args["columns"] = [col.strip() for col in value.split(",")]
                    
                    if matched_func and matched_func in tool_functions:
                        result = tool_functions[matched_func](**parsed_args)
                        tool_results.append({
                            "function": func_name,
                            "matched_function": matched_func,
                            "arguments": parsed_args,
                            "result": result
                        })
                    else:
                        tool_results.append({
                            "function": func_name,
                            "error": f"Bilinmeyen fonksiyon: {func_name}",
                            "arguments": args
                        })
                except Exception as e:
                    tool_results.append({
                        "function": getattr(call, "function", {}).get("name", "bilinmeyen"),
                        "error": f"Tool çağrısı işlenirken hata: {e}"
                    })
        
        return {"model_content": content, "tool_results": tool_results, "raw_response": response}
    except Exception as e:
        return {
            "error": f"Ollama ile iletişim hatası: {str(e)}",
            "model_content": None,
            "tool_results": [],
            "raw_response": None
        }

## Örnek: Ollama ile Tool Çağrısı ve Sonuçların Otomatik İşlenmesi

In [5]:
# Tool fonksiyonlarını harita olarak hazırla
tool_functions = {
    "yfinance_tool": lambda symbol, period="5d", columns=None: get_stock_data(symbol, period, columns)
}

# Tool şemasını Ollama'ya verilecek şekilde listele
tools = [yfinance_tool_schema]

prompt = "AAPL hissesinin son 1 günlük kapanış ve hacim verilerini getirir misin?\nLütfen tool'u kullan ve sadece kapanış (Close) ile hacim (Volume) kolonlarını getir."

print(f"Sorgu gönderiliyor: '{prompt}'")

try:
    result = ollama_chat_with_tools(
        prompt,
        tools,
        tool_functions
    )
    
    if "error" in result:
        print(f"❌ Hata: {result['error']}")
    
    print("\n📝 Model cevabı:")
    print(result["model_content"] if result["model_content"] else "(Boş içerik)")
    
    print("\n🔧 Tool sonuçları:")
    from pprint import pprint
    pprint(result["tool_results"])
    
    # Tool çağrısı gelmediyse, elle tool fonksiyonunu çalıştır (örnek, fallback)
    if not result["tool_results"]:
        print("\n⚠️ Otomatik tool çağrısı gelmedi. Elle tool fonksiyonunu çalıştırılıyor...")
        fallback = get_stock_data("AAPL", "1d", columns=["Close", "Volume"])
        print("\n📈 Elle çalıştırılan tool sonucu:")
        pprint(fallback)
except Exception as e:
    print(f"❌ İşlem hatası: {e}")
    # Herhangi bir hata durumunda fallback'i tanımla
    fallback = get_stock_data("AAPL", "1d", columns=["Close", "Volume"])

Sorgu gönderiliyor: 'AAPL hissesinin son 1 günlük kapanış ve hacim verilerini getirir misin?
Lütfen tool'u kullan ve sadece kapanış (Close) ile hacim (Volume) kolonlarını getir.'

📝 Model cevabı:
(Boş içerik)

🔧 Tool sonuçları:
[{'arguments': {'a': 'AAPL', 'd': '1d'},
  'error': 'Bilinmeyen fonksiyon: yq_d',
  'function': 'yq_d'}]

📝 Model cevabı:
(Boş içerik)

🔧 Tool sonuçları:
[{'arguments': {'a': 'AAPL', 'd': '1d'},
  'error': 'Bilinmeyen fonksiyon: yq_d',
  'function': 'yq_d'}]


## Sonuçların Görselleştirilmesi

In [ ]:
# Tool sonucu veya fallback ile gelen veriyi otomatik ve esnek şekilde görselleştirir
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

def plot_stock_data(stock_result, columns=None):
    """Veri sözlüğünden istenen kolonları çizer. columns=None ise tüm sayısal kolonları çizer."""
    if not stock_result:
        print("❌ Veri sözlüğü boş.")
        return
        
    if "error" in stock_result:
        print(f"❌ Veri hatası: {stock_result['error']}")
        return
        
    if "data" not in stock_result or not stock_result["data"]:
        print("❌ Veri bulunamadı.")
        return
        
    # DataFrame'e dönüştür
    df = pd.DataFrame(stock_result["data"])
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"])
        df.set_index("Date", inplace=True)
        
    if columns is None:
        # Sadece sayısal kolonları çiz
        columns = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col])]
        
    if not columns or not any(col in df.columns for col in columns):
        print(f"❌ Çizilecek kolonlar bulunamadı: {columns}")
        print(f"✓ Mevcut kolonlar: {list(df.columns)}")
        return
        
    # Eğer hem fiyat hem de hacim gösteriliyorsa, bunları ayrı y eksenlerinde çiz
    if "Volume" in columns and any(col in columns for col in ["Close", "Open", "High", "Low", "Adj Close"]):
        price_cols = [col for col in columns if col != "Volume"]
        
        fig, ax1 = plt.subplots(figsize=(12, 6))
        
        # Fiyat serilerini çiz
        for col in price_cols:
            if col in df.columns:
                ax1.plot(df.index, df[col], label=col)
                
        ax1.set_xlabel("Tarih")
        ax1.set_ylabel("Fiyat (USD)")
        ax1.tick_params(axis='y')
        
        # Hacim için ikinci y ekseni
        ax2 = ax1.twinx()
        if "Volume" in df.columns:
            ax2.bar(df.index, df["Volume"], alpha=0.3, color="gray", label="Volume")
            ax2.set_ylabel("Hacim")
            ax2.tick_params(axis='y')
            
        # Efsaneyi birleştir
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
        
        plt.title(f"{stock_result.get('symbol','')} Son {stock_result.get('period','')} Verileri")
    else:
        # Normal çizim - tek y ekseni
        valid_columns = [col for col in columns if col in df.columns]
        ax = df[valid_columns].plot(title=f"{stock_result.get('symbol','')} Son {stock_result.get('period','')} Verileri", figsize=(12, 6))
        ax.set_ylabel(", ".join(valid_columns))
        ax.set_xlabel("Tarih")
        plt.legend(valid_columns)
        
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Tool sonucu veya fallback'ten veri al
stock_result = None

# 1. Tool sonucunda veri varsa onu kullan
if "result" in globals() and isinstance(result, dict):
    if result.get("tool_results") and result["tool_results"]:
        for tool_result in result["tool_results"]:
            if "result" in tool_result and "error" not in tool_result["result"]:
                stock_result = tool_result["result"]
                break

# 2. Fallback'e bak
if stock_result is None and "fallback" in globals() and isinstance(fallback, dict):
    if "error" not in fallback:
        stock_result = fallback

# 3. Son çare data değişkenine bak
if stock_result is None and "data" in globals() and isinstance(data, pd.DataFrame):
    if not data.empty:
        # DataFrame'den sözlük formatına dönüştür
        stock_result = {
            "symbol": ticker if "ticker" in globals() else "AAPL",
            "period": "5d",
            "data": data.reset_index().to_dict(orient="records")
        }

# Veri var mı kontrol et
if stock_result:
    print(f"📊 {stock_result.get('symbol', 'Bilinmeyen')} hissesi için grafik çiziliyor...")
    # Kullanıcı isteğine göre sadece Close ve Volume çizilsin
    plot_stock_data(stock_result, columns=["Close", "Volume"])
else:
    print("❌ Görselleştirilecek veri bulunamadı.")

❌ Görselleştirilecek veri bulunamadı.


: 